#  References

Twitter API with Python: Part 1 -- Streaming Live Tweets, LucidProgramming
- https://www.youtube.com/watch?v=wlnx-7cm4Gg 

Twitter API with Python: Part 2 -- Cursor and Pagination  
- https://www.youtube.com/watch?v=rhBZqEWsZU4&list=PL5tcWHG-UPH2zBfOz40HSzcGUPAVOOnu1

Tweepy
- http://www.tweepy.org/
- http://docs.tweepy.org/en/v3.6.0/
- http://tweepy.readthedocs.io/en/v3.6.0/index.html


In [1]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [2]:
import config #located in same folder

consumer_key = config.twitter_anidata_consumer_key
consumer_secret = config.twitter_anidata_consumer_secret
access_token = config.twitter_anidata_access_token
access_token_secret = config.twitter_anidata_access_token_secret

In [3]:
# Create a class to print the tweets

class TwitterClient():
    
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth) #API is imported from tweepy
        
        self.twitter_user = twitter_user
        
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets
    
     def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets
        

class TwitterAuthenticator():
    '''
    Class for authenticating Twitter app
    '''
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)        # Create the authentication object
        auth.set_access_token(access_token, access_token_secret)  # Set access token and secret
        return auth      

    
class TwitterStreamer():
    
    """
    Class for streaming and processing live tweets
    """
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
    
    def stream_tweets(self, fetched_tweets_filename, search_terms):
        #This handles twitter authentication and the connection to the twitter streaming API
        
        listener = TwitterListener(fetched_tweets_filename) # Create and object of class TwitterListener
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)
        
        # filter the tweets using filter method provided by stream class
        stream.filter(track=search_terms)
        
        

class TwitterListener(StreamListener): # inherit from TwitterStreamer class which provides methods we can override

    def __init__(self, fetched_tweets_filename): #constructor
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):    # take in the data from the StreamListener, then we can do whatever we want with that data
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True             # return True to make sure everything went well
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True  
    
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)
        



In [4]:
if __name__ == "__main__":
    
    # Define search terms
    search_terms = ['#atlanta', '-filter:retweets']
    
    # Define output file name
    fetched_tweets_filename = "tweets.json"
    
    # Define number of tweets to collect
    number_of_tweets_to_return = 1
    
    # Optionally define a twitter user other than the application owner (owner's timeline)
    # If left empty, default to None and uses the application owner's timeline
    twitter_user = 'Ella__Dorsey'
    
    # Define TwitterStreamer object
    # twitter_streamer = TwitterStreamer()
    
    # twitter_streamer.stream_tweets(fetched_tweets_filename, search_terms)
    
    twitter_client = TwitterClient(twitter_user)
    print(twitter_client.get_user_timeline_tweets(number_of_tweets_to_return))
    

[Status(_api=<tweepy.api.API object at 0x000001C9BBAFC518>, _json={'created_at': 'Sun Jul 22 20:41:06 +0000 2018', 'id': 1021133056109948929, 'id_str': '1021133056109948929', 'text': 'RT @keeners1: @Ella__Dorsey Yesterday from Kroger parking lot in Versailles, KY. You know the rest. https://t.co/2PLwmVir4Z', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'keeners1', 'name': 'steve keener', 'id': 605629771, 'id_str': '605629771', 'indices': [3, 12]}, {'screen_name': 'Ella__Dorsey', 'name': 'Ella Dorsey', 'id': 362249392, 'id_str': '362249392', 'indices': [14, 27]}], 'urls': [], 'media': [{'id': 1020767447589085185, 'id_str': '1020767447589085185', 'indices': [100, 123], 'media_url': 'http://pbs.twimg.com/media/Dip-ldfUYAERzUl.jpg', 'media_url_https': 'https://pbs.twimg.com/media/Dip-ldfUYAERzUl.jpg', 'url': 'https://t.co/2PLwmVir4Z', 'display_url': 'pic.twitter.com/2PLwmVir4Z', 'expanded_url': 'https://twitter.com/keeners1/status/102076